Загрузка обработанного датасета с личного Google Drive.

In [1]:
import gdown
url = "https://drive.google.com/drive/folders/16xTuNcBlzLzxAXE25rYoKztghsdHKb--?usp=drive_link"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/prepared-data/cache-0bf1b118d60bcde3.arrow',
 '/content/prepared-data/cache-5e30699a7978c804.arrow',
 '/content/prepared-data/cache-6fa1062e4bd6a5e1.arrow',
 '/content/prepared-data/cache-7b4579fd319655b4.arrow',
 '/content/prepared-data/cache-8cffc1c9416249ea.arrow',
 '/content/prepared-data/cache-8ff8b28f0a40b846.arrow',
 '/content/prepared-data/cache-14b974e6761a1ee8.arrow',
 '/content/prepared-data/cache-20c9961dcb9aa7af.arrow',
 '/content/prepared-data/cache-21d8d0fffa7d3ef7.arrow',
 '/content/prepared-data/cache-38b420facd7b0911.arrow',
 '/content/prepared-data/cache-71ccfea339ebeda2.arrow',
 '/content/prepared-data/cache-84e7c6f4b3fd11cb.arrow',
 '/content/prepared-data/cache-157bb8a23015943f.arrow',
 '/content/prepared-data/cache-267e053652b84e01.arrow',
 '/content/prepared-data/cache-2364d3ceb9033ef0.arrow',
 '/content/prepared-data/cache-5197ae4d5164021a.arrow',
 '/content/prepared-data/cache-44294a79f8289f5b.arrow',
 '/content/prepared-data/cache-092542cc72b883c4.

In [5]:
!pip install --upgrade pip
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12==24.6.*

!pip install wandb datasets==2.20.0 evaluate==0.4.2 numpy==1.26.4 scikit-learn==1.5.1 tqdm==4.66.4 transformers torch openpyxl accelerate>=0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [6]:
import datasets
import evaluate
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import wandb

wandb.init(anonymous='must')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
TEST_SIZE_FRACTION = 0.1
OUTPUT_DIR = Path(f'/content/bert-model')

conf_matrix_metric = evaluate.load("confusion_matrix")
f1_metric = evaluate.load("f1")

def classifier(dataset: datasets.Dataset):
    # Load model directly
    tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    model = model.to("cuda:0")

    def tokenization_map(data):
        return tokenizer(data["message"], truncation=True, padding=True)
    mapped_dataset = dataset.map(tokenization_map, batched=True)

    mapped_dataset = mapped_dataset.rename_column("is_toxic", "labels")
    train_and_test = mapped_dataset.train_test_split(test_size=TEST_SIZE_FRACTION)

    def compute_metrics(results):
        y_pred, y = results
        y_pred = y_pred.argmax(axis=1)
        f1 = f1_metric.compute(predictions=y_pred, references=y, average='weighted')
        conf_matrix = conf_matrix_metric.compute(predictions=y_pred, references=y)
        return {'f1': f1['f1'], "conf_matrix": conf_matrix["confusion_matrix"]}

    options = TrainingArguments(
        output_dir=OUTPUT_DIR,
        eval_strategy="epoch",
        learning_rate=1e-4,
        num_train_epochs=1,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        remove_unused_columns=True,
    )
    trainer = Trainer(
        model=model,
        args=options,
        train_dataset=train_and_test["train"],
        eval_dataset=train_and_test["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    stats = trainer.evaluate()
    print(stats)

In [15]:
DEFAULT_PREP_DATASET_PATH = Path('/content/prepared-data')

def load_dataset(path: Path) -> datasets.Dataset:
    return datasets.load_from_disk(str(path))

classifier(load_dataset(DEFAULT_PREP_DATASET_PATH))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Conf Matrix
1,0.322000,0.236498,0.926356,[[1000 47] [ 48 196]]


Trainer is attempting to log a value of "[[1000   47]
 [  48  196]]" of type <class 'numpy.ndarray'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "[[1000   47]
 [  48  196]]" of type <class 'numpy.ndarray'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.23649752140045166, 'eval_f1': 0.9263556445365877, 'eval_conf_matrix': array([[1000,   47],
       [  48,  196]]), 'eval_runtime': 41.1379, 'eval_samples_per_second': 31.382, 'eval_steps_per_second': 1.969, 'epoch': 1.0}
